In [ ]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

#subroutine to read in all experiments from single excel file
def readData(fname):
    xl = pd.ExcelFile(fname)
    sheet_names = xl.sheet_names #)  # see all sheet names
    #print(sheet_names)
    istart=0
    for sheet_name in sheet_names:
        if sheet_name=='Metadata':
            continue
        ds1 = xl.parse(sheet_name,header=0,skiprows=1)
        if istart==0:
            ds=ds1
            istart=1
        else:
            ds=pd.concat([ds,ds1]) #ds.append(ds1)
        #print(type(ds1))
        #print(type(ds))
        print(sheet_name,len(ds))
    return ds


In [ ]:
adir = 'C:/Users/gentemann/Google Drive/f_drive/docs/papers/in_prep/2018dave/'
#file = adir + 'Master_Data_Jump_Test_2014_2017_2018.xlsx' #'Duplicate_Master_Data_Jump_Test_2014_2017_2018.xlsx'
file = adir + 'Master_Data_Jump_Test_2014_2017_2018_2019_4JUN19.xlsx' #'Duplicate_Master_Data_Jump_Test_2014_2017_2018.xlsx'
ds2 = readData(file)
df2 = pd.DataFrame(ds)
#file = adir + 'Master_Data_Jump_Test_2014_2017_2018_2019_19SEPT19.xlsx'
file = adir + 'Master_Data_Jump_Test_2014_2017_2018_2019_12NOV19.xlsx'
ds = readData(file)
df = pd.DataFrame(ds)


In [ ]:
df.head()

In [ ]:
ds.water_temperature.hist(bins=np.arange(9,16))

In [ ]:
ds2.water_temperature.hist(bins=np.arange(9,16))

In [ ]:
ds[285:300]

In [ ]:
#get some info about the data
#print out number of fish per experiment for table on data
print(ds.groupby('Experiment_number').fish_number.max())
cond_6 = (ds.height_difference<8)  #6" jumps
cond_12 = (ds.height_difference>8) & (ds.height_difference<15)  #12" jumps
cond_18 = (ds.height_difference>15)  #18" jumps
subset = ds[cond_6]
print('6',subset.groupby('Experiment_number').fish_number.max())
subset = ds[cond_12]
print('12',subset.groupby('Experiment_number').fish_number.max())
subset = ds[cond_18]
print('18',subset.groupby('Experiment_number').fish_number.max())


In [ ]:
# the histogram of the data
#fig = plt.figure()
#n, bins, patches = plt.hist(subset_lower.k_factor, 50, normed=1, facecolor='g', alpha=0.75)
#plt.xlabel('k_factor')
#plt.ylabel('Probability')
#plt.title('Histogram of k_factor')
#plt.text(60, .025, r'$\mu=100,\ \sigma=15$')
#plt.axis([0, 2, 0, 0.03])
#plt.grid(True)
#fig.savefig(adir+'hist.png', dpi=300)

In [ ]:
#plot all fish
cond_lower = (ds.successful_jump==0)   # stayed in lower pool, unsuccessful jumper
cond_upper = (ds.successful_jump==1)  #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 245,0, 140])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'all_scatter_10082019.png', dpi=300)

In [ ]:
#6" jumps
#cond_lower = ((ds.successful_jump==0) & (ds.height_difference<10))  # stayed in lower pool, unsuccessful jumper
#cond_upper = ((ds.successful_jump==1) & (ds.height_difference<10)) #made it to upper pool
cond_lower = ((ds.successful_jump==0) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 200,0, 70])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'six_scatter_10082019.png', dpi=300)

In [ ]:
#12" jumps
cond_lower = ((ds.successful_jump==0) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 200,0, 70])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'twelve_scatter_10082019.png', dpi=300)

In [ ]:
#12" jumps
cond_lower = ((ds.successful_jump==0) & cond_18)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_18) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
fig = plt.figure()
h=plt.scatter(subset_lower.length,subset_lower.weight,color='r',marker='.',s=12,label="lower")
h=plt.scatter(subset_upper.length,subset_upper.weight,color='k',marker='.',s=12,label="upper")
plt.axis([0, 240,0, 130])
plt.ylabel('Weight (g)')
plt.xlabel('Length (mm)')
plt.legend()
fig.savefig(adir+'eighteen_scatter_10082019.png', dpi=300)
print(cond_lower.sum()+cond_upper.sum())
print(ds.length.max(),ds.weight.max())

In [ ]:
#for 6" jump, 12 18
subset = ds[cond_18]
cols = subset.columns.tolist()
print(cols)
cols_reorder = cols[12:13] + cols[15:16] + cols[14:15]
df_reorder=subset[cols_reorder]
df_reorder.head()
x = df_reorder.iloc[:,0:1].values  #just length  0.79 accuracy new with all data 0.71
#x = df_reorder.iloc[:,1:2].values    #just k-factor  0.71 accuracy new with all data 0.56
#x = df_reorder.iloc[:,0:2].values     #both length and k-factor but worried about their correlation  0.78 accuracy, new 0.74
y= df_reorder.iloc[:,2].values
# Split the data into Training and Testing set
#from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=0)
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)
#Fitting logistic regression to the training set
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(x_train,y_train)
# Predicting the Test set results
y_pred = classifier.predict(x_test)
# Making the confusion matrix 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
from sklearn import metrics
print('18"')
print ('accuracy score = ',metrics.accuracy_score(y_test,classifier.predict(x_test)))
print('confusion matrix = ',cm)
print('coef=',classifier.coef_)

In [ ]:
classifier.coef_

In [ ]:
cm

In [ ]:
#make figure showing prob of successful jump for different jump heights
fish_num_cutoff = 5
fish_length_int=ds.length.values.astype(int)
width_bin=1
hbins = np.arange(0,200,width_bin)-.5
cond_lower = ((ds.successful_jump==0) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6 = hist_jump / tot_fish
prob_jump_6[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12 = hist_jump / tot_fish
prob_jump_12[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_18)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_18) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_18 = hist_jump / tot_fish
prob_jump_18[cond_num]=np.nan
fig = plt.figure()
hfish,hbin2=np.histogram(fish_length_int[cond_6],bins=hbins)
plt.bar(bin_edges[:-1],hfish/hfish.max(),width=width_bin, color='r', alpha=0.5)
hfish,hbin2=np.histogram(fish_length_int[cond_12],bins=hbins)
plt.bar(bin_edges[:-1],hfish/hfish.max(), width=width_bin,color='b', alpha=0.5)
hfish,hbin2=np.histogram(fish_length_int[cond_18],bins=hbins)
plt.bar(bin_edges[:-1],hfish/hfish.max(),width=width_bin, color='k', alpha=0.5)
plt.plot(bin_edges[:-1],prob_jump_6, color='r')
plt.plot(bin_edges[:-1],prob_jump_12, color='b')
plt.plot(bin_edges[:-1],prob_jump_18, color='k')
plt.axis([0, 200,0, 1])
plt.ylabel('Probability Successful Jump (%)')
plt.xlabel('Length (mm)')
plt.text(10,.9,'6" jump',color='r')
plt.text(10,.84,'12" jump',color='b')
plt.text(10,.78,'18" jump',color='k')
fig.savefig(adir+'prob_1mmbins_10082019.png', dpi=300)
    

In [ ]:
#prob successful jump, in 5mm bins rather than 1mm bins
fish_num_cutoff = 10
fish_length_int=ds.length.values.astype(int)
width_bin=5
hbins = np.arange(0,200,width_bin)-.5
cond_lower = ((ds.successful_jump==0) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6 = hist_jump / tot_fish
prob_jump_6[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12 = hist_jump / tot_fish
prob_jump_12[cond_num]=np.nan
cond_lower = ((ds.successful_jump==0) & cond_18)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & cond_18) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_18 = hist_jump / tot_fish
prob_jump_18[cond_num]=np.nan
fig = plt.figure()
hfish,hbin2=np.histogram(fish_length_int[cond_6],bins=hbins)
plt.bar(bin_edges[:-1],hfish/hfish.max(), width=width_bin,color='r', alpha=0.5)
hfish,hbin2=np.histogram(fish_length_int[cond_12],bins=hbins)
plt.bar(bin_edges[:-1],hfish/hfish.max(), width=width_bin,color='b', alpha=0.5)
hfish,hbin2=np.histogram(fish_length_int[cond_18],bins=hbins)
plt.bar(bin_edges[:-1],hfish/hfish.max(), width=width_bin,color='k', alpha=0.5)
plt.plot(bin_edges[:-1],prob_jump_6, color='r')
plt.plot(bin_edges[:-1],prob_jump_12, color='b')
plt.plot(bin_edges[:-1],prob_jump_18, color='k')
plt.axis([0, 200,0, 1])
plt.ylabel('Probability Successful Jump (%)')
plt.xlabel('Length (mm)')
plt.text(10,.9,'6" jump',color='r')
plt.text(10,.84,'12" jump',color='b')
plt.text(10,.78,'18" jump',color='k')
fig.savefig(adir+'prob_'+str(width_bin)+'mmbins_10082019.png', dpi=300)

In [ ]:
#hbins = np.arange(0,200,10)-.5
#cond_lower = ((ds.successful_jump==0) & cond_6)  # stayed in lower pool, unsuccessful jumper
#cond_upper = ((ds.successful_jump==1) & cond_6) #made it to upper pool
ds.length[cond_18].hist(bins=range(0,200,5))


In [ ]:
ds.length.hist(bins=np.arange(0,200,10))

In [ ]:
#(ds.water_temperature*(9/5)+32).hist(bins=np.arange(45.25,55,.5))
ds.water_temperature.hist(bins=np.arange(8.25,15,.5))

In [ ]:
#prob successful jump, in 5mm bins rather than 1mm bins
fish_num_cutoff = 5
fish_length_int=ds.length.values.astype(int)
width_bin=5
hbins = np.arange(0,200,width_bin)-.5

#6" cold
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature<11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature<11) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6 = hist_jump / tot_fish
prob_jump_6[cond_num]=np.nan

#6" warm
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature>11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature>11) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6_warm = hist_jump / tot_fish
prob_jump_6_warm[cond_num]=np.nan

#12" cold
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature<11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature<11) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12 = hist_jump / tot_fish
prob_jump_12[cond_num]=np.nan

#12" warm
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature>11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature>11) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12_warm = hist_jump / tot_fish
prob_jump_12_warm[cond_num]=np.nan

fig = plt.figure()
hfish,hbin2=np.histogram(fish_length_int[cond_6],bins=hbins)
#plt.bar(bin_edges[:-1],hfish/hfish.max(), width=width_bin,color='r', alpha=0.5)
hfish,hbin2=np.histogram(fish_length_int[cond_12],bins=hbins)
#plt.bar(bin_edges[:-1],hfish/hfish.max(), width=width_bin,color='b', alpha=0.5)
plt.plot(bin_edges[:-1],prob_jump_6, color='r')
plt.plot(bin_edges[:-1],prob_jump_6_warm, color='m')
plt.plot(bin_edges[:-1],prob_jump_12, color='b')
plt.plot(bin_edges[:-1],prob_jump_12_warm, color='c')
plt.axis([0, 200,0, 1])
plt.ylabel('Probability Successful Jump (%)')
plt.xlabel('Length (mm)')


ystart,voff=.9,0.06
plt.text(155,ystart-0*voff,'54$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-0*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-1*voff,'6" jump',color='m',fontsize=12)
plt.text(155,ystart-2*voff,'12" jump',color='c',fontsize=12)
plt.text(155,ystart-8*voff,'49$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-8*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-9*voff,'6" jump',color='r',fontsize=12)
plt.text(155,ystart-10*voff,'12" jump',color='b',fontsize=12)

fig.savefig(adir+'prob_'+str(width_bin)+'mmbins_temp_10082019.png', dpi=300)

In [ ]:
#6" jump: 2018-2 (17 fish), 2017-17 (3 fish), 2017-18 (2 fish)
#12" jump:  2018-1(6 fish), 2017-20 (7 fish) 
#prob successful jump, in 5mm bins rather than 1mm bins
fish_num_cutoff = 5
fish_length_int=ds.length.values.astype(int)
width_bin=5
hbins = np.arange(0,200,width_bin)-.5

#6" cold
cond_6_cold = ((ds.water_temperature<11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature<11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature<11) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6 = hist_jump / tot_fish
prob_jump_6[cond_num]=np.nan

#6" warm
cond_6_warm = ((ds.water_temperature>11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature>11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature>11) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6_warm = hist_jump / tot_fish
prob_jump_6_warm[cond_num]=np.nan

#12" cold
cond_12_cold = ((ds.water_temperature<11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature<11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature<11) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12 = hist_jump / tot_fish
prob_jump_12[cond_num]=np.nan

#12" warm
cond_12_warm = ((ds.water_temperature>11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature>11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature>11) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12_warm = hist_jump / tot_fish
prob_jump_12_warm[cond_num]=np.nan

fig = plt.figure(figsize=(6,8))
plt.subplot(211)
plt.plot(bin_edges[:-1],prob_jump_6, color='r')
plt.plot(bin_edges[:-1],prob_jump_6_warm, color='m')
plt.plot(bin_edges[:-1],prob_jump_12, color='b')
plt.plot(bin_edges[:-1],prob_jump_12_warm, color='c')
plt.axis([0, 200,0, 1])
plt.ylabel('Probability Successful Jump (%)')
plt.xlabel('Length (mm)')
ystart,voff=.9,0.06
plt.text(155,ystart-0*voff,'54$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-0*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-1*voff,'6" jump',color='m',fontsize=12)
plt.text(155,ystart-2*voff,'12" jump',color='c',fontsize=12)
plt.text(155,ystart-8*voff,'49$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-8*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-9*voff,'6" jump',color='r',fontsize=12)
plt.text(155,ystart-10*voff,'12" jump',color='b',fontsize=12)

plt.subplot(212)
plt.hist(fish_length_int[cond_6_cold],bins=hbins,color='r', histtype='step')
plt.hist(fish_length_int[cond_12_cold],bins=hbins,color='b', histtype='step')
plt.hist(fish_length_int[cond_6_warm],bins=hbins,color='m', histtype='step')
plt.hist(fish_length_int[cond_12_warm],bins=hbins,color='c', histtype='step')
plt.axis([0, 200,0, 235])
plt.ylabel('Number of Fish')
plt.xlabel('Length (mm)')

ystart,voff=220,15
plt.text(155,ystart-0*voff,'54$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-0*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-1*voff,'6" jump',color='m',fontsize=12)
plt.text(155,ystart-2*voff,'12" jump',color='c',fontsize=12)
plt.text(155,ystart-8*voff,'49$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-8*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-9*voff,'6" jump',color='r',fontsize=12)
plt.text(155,ystart-10*voff,'12" jump',color='b',fontsize=12)



fig.savefig(adir+'prob_'+str(width_bin)+'mmbins_temp2_hist.png', dpi=300)


In [ ]:
#prob successful jump, in 5mm bins rather than 1mm bins
fish_num_cutoff = 5
fish_length_int=ds.length.values.astype(int)
width_bin=5
hbins = np.arange(0,200,width_bin)-.5

#6" cold
cond_6_cold = ((ds.water_temperature<11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature<11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature<11) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6 = hist_jump / tot_fish
prob_jump_6[cond_num]=np.nan

#6" warm
cond_6_warm = ((ds.water_temperature>11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature>11) & cond_6)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature>11) & cond_6) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_6_warm = hist_jump / tot_fish
prob_jump_6_warm[cond_num]=np.nan

#12" cold
cond_12_cold = ((ds.water_temperature<11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature<11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature<11) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12 = hist_jump / tot_fish
prob_jump_12[cond_num]=np.nan

#12" warm
cond_12_warm = ((ds.water_temperature>11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_lower = ((ds.successful_jump==0) & (ds.water_temperature>11) & cond_12)  # stayed in lower pool, unsuccessful jumper
cond_upper = ((ds.successful_jump==1) & (ds.water_temperature>11) & cond_12) #made it to upper pool
subset_upper = ds[cond_upper]
subset_lower = ds[cond_lower]
hist_jump, bin_edges = np.histogram(fish_length_int[cond_upper],bins=hbins)
hist_nojump, bin_edges = np.histogram(fish_length_int[cond_lower],bins=hbins)
tot_fish = hist_jump + hist_nojump
cond_num = (tot_fish < fish_num_cutoff)
prob_jump_12_warm = hist_jump / tot_fish
prob_jump_12_warm[cond_num]=np.nan

fig = plt.figure(figsize=(6,8))
plt.subplot(211)
plt.plot(bin_edges[:-1],prob_jump_6, color='r')
plt.plot(bin_edges[:-1],prob_jump_6_warm, color='m')
plt.plot(bin_edges[:-1],prob_jump_12, color='b')
plt.plot(bin_edges[:-1],prob_jump_12_warm, color='c')
plt.axis([0, 200,0, 1])
plt.ylabel('Probability Successful Jump (%)')
plt.xlabel('Length (mm)')
ystart,voff=.9,0.06
plt.text(155,ystart-0*voff,'54$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-0*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-1*voff,'6" jump',color='m',fontsize=12)
plt.text(155,ystart-2*voff,'12" jump',color='c',fontsize=12)
plt.text(155,ystart-8*voff,'49$^{\circ}$F water',color='black',fontsize=12)
plt.text(155,ystart-8*voff,'___________',color='black',fontsize=12)
plt.text(155,ystart-9*voff,'6" jump',color='r',fontsize=12)
plt.text(155,ystart-10*voff,'12" jump',color='b',fontsize=12)

plt.subplot(212)
hfish,hbin2=np.histogram(fish_length_int[cond_6_cold],bins=hbins)
#plt.bar(bin_edges[:-1],hfish, width=width_bin,color='w',edgecolor='r', alpha=0.8)
plt.plot(bin_edges[:-1],hfish, color='r')
#plt.bar(bin_edges[:-1],hfish/hfish.max(), width=width_bin,color='r', alpha=0.3)
hfish,hbin2=np.histogram(fish_length_int[cond_12_warm],bins=hbins)
plt.plot(bin_edges[:-1],hfish, color='c')
#plt.bar(bin_edges[:-1],hfish, width=width_bin,color='w',edgecolor='c', alpha=0.8)
hfish,hbin2=np.histogram(fish_length_int[cond_6_warm],bins=hbins)
plt.plot(bin_edges[:-1],hfish, color='m')
#plt.bar(bin_edges[:-1],hfish, width=width_bin,color='w',edgecolor='m', alpha=0.8)
hfish,hbin2=np.histogram(fish_length_int[cond_12_cold],bins=hbins)
plt.plot(bin_edges[:-1],hfish, color='b')
#plt.bar(bin_edges[:-1],hfish, width=width_bin,color='w',edgecolor='b', alpha=0.8)
plt.axis([0, 200,0, 310])
plt.ylabel('Number of Fish')
plt.xlabel('Length (mm)')
plt.legend('')


fig.savefig(adir+'prob_'+str(width_bin)+'mmbins_temp2_hista_10082019.png', dpi=300)

In [ ]:
print(sum(cond_6),sum(cond_12),sum(cond_6)+sum(cond_12)+sum(cond_18))

In [ ]:
print(ds[cond_6_warm & (fish_length_int>=105)].shape)
print(ds[cond_12_warm & (fish_length_int>=105)].shape)
print(ds[cond_6_cold & (fish_length_int>=105)].shape)
print(ds[cond_12_cold & (fish_length_int>=105)].shape)

In [ ]:
(ds.species=='Coho').sum()

In [ ]:
hfish,hbin2=np.histogram(fish_length_int[cond_6_cold],bins=hbins)


In [ ]:
for i in range(100):
    print(hfish[i],hbin2[i])